# Import backage

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from train import train_plus
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from vector_field import mesh
from vector_field import transit
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Generate data

In [3]:
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 0.01
interval = 100
steps2=10
seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10*interval, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt,interval=interval) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps*interval, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt,interval=interval)

# Train

In [ ]:
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001
eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train_plus(train_data=train_data, test_data=test_data, 
                                                                         sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                         mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis',steps2=steps2,interval=interval)
torch.save(net.state_dict(), 'mdl_data/nis steps10 exp-0.2t interval100.mdl')

check point-------o_0-> 12:08:11.490594; lasting 0:00:05.559386 seconds
Epoch: 0
Train loss: 0.2162
dEI: 0.5717
term1: -0.2794
term2: 1.4228
Test multistep loss: 0.4427
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 12:08:17.242163; lasting 0:00:05.751569 seconds
check point-------o_0-> 12:10:08.984880; lasting 0:01:51.742717 seconds
Epoch: 500
Train loss: 0.0276
dEI: 0.9734
term1: 0.6366
term2: 1.3102
Test multistep loss: 0.2109
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 12:10:18.159090; lasting 0:00:09.174210 seconds
check point-------o_0-> 12:12:09.068718; lasting 0:01:50.909628 seconds
Epoch: 1000
Train loss: 0.0167
dEI: 1.3832
term1: 1.4307
term2: 1.3357
Test multistep loss: 0.2103
--------------------------------------------------------------------------------------------------------

In [ ]:
eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net_nisp = train_plus(train_data=train_data, test_data=test_data, 
                                                                         sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                         mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis+',steps2=steps2,interval=interval)
torch.save(net_nisp.state_dict(), 'mdl_data/nisp steps10 exp-0.2t interval100.mdl')

check point-------o_0-> 14:56:39.694001; lasting 0:44:38.070512 seconds
Epoch: 0
Train loss: 0.2350
dEI: 0.5724
term1: -0.2786
term2: 1.4234
Test multistep loss: 0.4411
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 14:56:45.977337; lasting 0:00:06.283336 seconds
check point-------o_0-> 14:59:04.215926; lasting 0:02:18.238589 seconds
Epoch: 500


In [ ]:

def SIR(s,i,lam=1,miu=0.5):
    ds=-lam * s * i
    di=lam * s * i - miu * i
    return ds, di
sz = 4
scale = 2
L = 1
sigma=0.03
rou=-0.5
hidden_units = 64
net = Renorm_Dynamic(sym_size = sz, latent_size = scale, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
net2 = Renorm_Dynamic(sym_size = sz, latent_size = scale, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
net.to(device=device)
net2.to(device=device)
net.load_state_dict(torch.load('nis steps10 exp-0.2t interval100.mdl'))
net2.load_state_dict(torch.load('net steps10 exp-0.2t interval100.mdl'))
spring = Simple_Spring_Model(device=device)

colorlabel = ['#F4F1DE','#DF7A5E','#3C405B','#82B29A','#F2CC8E']
colors=[colorlabel[1]]*len(S)+[colorlabel[3]]*len(S)
#0.007,0.015,0.07


dd1_nis,dd2_nis=vector_func(net,jac_bool=False)
#dd1_nis_jac,dd2_nis_jac=vector_func(net,jac_bool=True)
#dd1_net,dd2_net=vector_func(net2,jac_bool=False)
#dd1_net_jac,dd2_net_jac=vector_func(net2,jac_bool=True)

In [ ]:
def SIR(s,i,lam=1,miu=0.5):
    ds=-lam * s * i
    di=lam * s * i - miu * i
    return ds, di
S, I = mesh(0.07)
dSdt,dIdt = SIR(S,I)
m=0.1
dSdt,dIdt=m*dSdt,m*dIdt
fig,ax = plt.subplots(figsize=(5,4),dpi=150)
colors=[colorlabel[2]]*len(S)+[colorlabel[3]]*len(S)+[colorlabel[1]]*len(S)
S=list(S)+list(S)+list(S)
I=list(I)+list(I)+list(I)
dd1_final=list(dd1_net)+list(dd1_net_jac)+list(dSdt)
dd2_final=list(dd2_net)+list(dd2_net_jac)+list(dIdt)
# 绘制拼接后的向量场
quiver = ax.quiver(S, I, dd1_final, dd2_final, width=0.005,color=colors)
#quiver = ax.quiver(S, I, dd1_final_nis, dd2_final_nis, width=0.005,color=colors)
red_patch = matplotlib.patches.Patch(color=colorlabel[1], label='real')
green_patch = matplotlib.patches.Patch(color=colorlabel[3], label='NIS+')
ran_patch = matplotlib.patches.Patch(color=colorlabel[2], label='theoretic')
plt.legend(handles=[red_patch, green_patch,ran_patch],loc='best', fontsize=12)

# Set graphic title and axis labels.
plt.xlabel('$S(h_1)$',fontsize=15)
plt.ylabel('$I(h_2)$',fontsize=15)

ax.tick_params(labelsize=15)
plt.show()

# Results

In [ ]:
plt.figure(dpi=300)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, eis_nis, label='NIS')
plt.plot(timeseries, eis_nisp, label='NIS+')
plt.title(r'$\mathcal{J}$')
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=300)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, losses_nis, label='NIS')
plt.plot(timeseries, losses_nisp, label='NIS+')
plt.title(r'loss')
plt.legend()
#plt.yscale('log')
plt.show()

In [ ]:
plt.figure(dpi=300)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, MAEs_mstep_nis, label='NIS')
plt.plot(timeseries, MAEs_mstep_nisp, label='NIS+')
plt.title(r'multistep mae')
plt.legend()
plt.show()